In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [2]:
!pip install transformers datasets sentencepiece sacrebleu
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-c

In [3]:
import pandas as pd
from datasets import Dataset

# Load your dataset (adjust separator if needed)
df = pd.read_csv("/kaggle/input/phones-csv/phone_en2hi_translation.csv")
# Drop rows with missing translations
df = df[['phones', 'hi_text']].dropna()

# Rename columns for compatibility
df = df.rename(columns={'phones': 'translation_en', 'hi_text': 'translation_hi'})

# Convert to HuggingFace dataset
dataset = Dataset.from_pandas(df)

# Format as translation pair
dataset = dataset.map(lambda x: {'translation': {'en': x['translation_en'], 'hi': x['translation_hi']}})

Map:   0%|          | 0/9248 [00:00<?, ? examples/s]

In [4]:
dataset

Dataset({
    features: ['translation_en', 'translation_hi', 'translation'],
    num_rows: 9248
})

In [5]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

# Set tokenizer for English → Hindi
tokenizer.src_lang = "en_XX"
tokenizer.tgt_lang = "hi_IN"

2025-04-14 21:59:03.245404: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744667943.437167      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744667943.495776      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [6]:
def preprocess_function(examples):
    # Iterate over the list of dictionaries for the "translation" column
    inputs = [trans["en"] for trans in examples["translation"]]
    targets = [trans["hi"] for trans in examples["translation"]]
    
    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    
    # Tokenize the targets in the target context
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/9248 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [7]:
tokenized_dataset

Dataset({
    features: ['translation_en', 'translation_hi', 'translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 9248
})

In [8]:
import wandb
wandb.login(key="a99e291f456a7abc8c5633970e998e968c1ef8b2")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: amushuk890 (amushuk890-iiit-hyderabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [9]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
!pip install evaluate
import evaluate
from transformers import Seq2SeqTrainer
import numpy as np

# Load the BLEU metric
bleu_metric = evaluate.load("bleu")

def compute_metrics(eval_preds):
    predictions, labels = eval_preds

    # Decode the predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Prepare references as a list of lists
    references = [[label] for label in decoded_labels]

    # Compute BLEU score
    result = bleu_metric.compute(predictions=decoded_preds, references=references)

    # Return the BLEU score
    return {"bleu": result["bleu"]}


split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print(f"Train dataset size: {len(train_dataset)}")
print(f"Test (Eval) dataset size: {len(eval_dataset)}")
training_args = Seq2SeqTrainingArguments(
    output_dir="./mbart50-hi",
    eval_strategy="epoch",         # Evaluate at the end of each epoch
    save_strategy="epoch",         # Save a checkpoint at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=1,            # Keep only the most recent checkpoint
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

from transformers import TrainerCallback
import random

import csv
from datetime import datetime

class InferenceCallback(TrainerCallback):
    def __init__(self, tokenizer, model, eval_dataset, num_samples=10, output_file="inference_samples.csv"):
        self.tokenizer = tokenizer
        self.model = model
        self.eval_dataset = eval_dataset
        self.num_samples = num_samples
        self.output_file = output_file
        self.sample_indices = random.sample(range(len(self.eval_dataset)), self.num_samples)
        self.sample_texts = [self.eval_dataset[i]['translation']['en'] for i in self.sample_indices]
        self.references = [self.eval_dataset[i]['translation']['hi'] for i in self.sample_indices]
        self.outputs = []

    def on_epoch_end(self, args, state, control, **kwargs):
        # Prepare inputs
        inputs = self.tokenizer(self.sample_texts, padding=True, truncation=True, return_tensors="pt").to(self.model.device)
        generated_ids = self.model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
        predictions = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        # Save this epoch's outputs for final CSV
        for src, ref, pred in zip(self.sample_texts, self.references, predictions):
            self.outputs.append({
                "epoch": state.epoch,
                "source": src,
                "reference": ref,
                "prediction": pred
            })

        # Console output
        print(f"\n--- Inference After Epoch {state.epoch:.1f} ---")
        for src, pred, ref in zip(self.sample_texts, predictions, self.references):
            print(f"Source: {src}")
            print(f"Predicted: {pred}")
            print(f"Reference: {ref}")
            print("------")
        print("\n")

        return control

    def on_train_end(self, args, state, control, **kwargs):
        # Write all accumulated outputs to CSV at the end of training
        keys = ["epoch", "source", "reference", "prediction"]
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_csv = f"inference_samples_{timestamp}.csv"

        with open(output_csv, "w", encoding="utf-8", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=keys)
            writer.writeheader()
            writer.writerows(self.outputs)

        print(f"Inference samples saved to {output_csv}")



# Instantiate the custom inference callback
inference_callback = InferenceCallback(
    tokenizer=tokenizer,
    model=model,
    eval_dataset=eval_dataset,
    num_samples=10  # You can adjust this to 3 or 4 samples
)

# Create the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Using the separate test set (1% of original data)
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[inference_callback]  # Add the inference callback here
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00


Train dataset size: 8323
Test (Eval) dataset size: 925


/tmp/ipykernel_19/509891933.py:117: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [10]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Tracking run with wandb version 0.19.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250414_220001-xatqayb7
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ./mbart50-hi
wandb: ⭐️ View project at https://wandb.ai/amushuk890-iiit-hyderabad/huggingface
wandb: 🚀 View run at https://wandb.ai/amushuk890-iiit-hyderabad/huggingface/runs/xatqayb7


Epoch,Training Loss,Validation Loss,Bleu
1,0.615300,0.552893,0.101132
2,0.465100,0.486348,0.153254
3,0.366900,0.467530,0.179064
4,0.291000,0.475744,0.189044
5,0.230100,0.489769,0.194732
6,0.180000,0.500066,0.199261
7,0.127300,0.515855,0.196542
8,0.101100,0.525438,0.205702
9,0.087200,0.530531,0.205340
10,0.075900,0.537429,0.203945


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



--- Inference After Epoch 1.0 ---
Source: d̪ ɪ s  ʋ iː k  j ʉ ʋ ɪ l l ɜː n ə b aw ʈ ʈ ɛ c ɲ ɪ k ə l ɪ n f ɹ ə s ʈ ɹ ə k tʃ ə  a n ɖ ɑː c ɪ ʈ ɛ k tʃ ə ɒ ʋ ɖ ɪ dʒ ɪ ʈ ə l l aj b ɾ i ɪ n ɖ iː ʈ eː l  
Predicted: इस सप्ताह में, आप डिजिटल लाइब्रेरी के डिजिटल इंफ्रास्ट्रक्चर और आर्किटेक्चर के बारे में सीखेंगे
Reference: इस हफ्ते, आप डिजिटल लाइब्रेरी की टेक्निकल इंफ्रास्ट्रक्चर और आर्किटेक्चर के बारे में विस्तार से जानेंगे
------
Source: d̪ ə spn h a z ɒː l s oː ɖ ɪ ʋ ɛ l ə p ʈ ə ɲ ʉː m ɛ ʈ ə ɖ eː ʈ ə s ʈ a n ɖ ɪ ɖ  k ɒː l ɖ spn d̪ a ʈ ɪ z  ɛ ʎ ɪ k ʈ ɹ a ɲ ɪ k  spn  ɖ ɪ s ə ʈ eː ʃ ə n m ɛ ʈ ə ɖ eː ʈ ə  s ʈ a n ɖ ɪ ɖ  
Predicted: आईओटी में मेटाडेटा स्टोरेज और एडेड कोड आईओटी भी विकसित किया गया है, जो इलेक्ट्रॉनिक सॉफ्टवेयर डिस्टेंस मेटाडेटा स्टोरेज और एडेड है
Reference: एनडीएलटीडी ने एक नया मेटाडेटा मानक भी विकसित किया है जिसे ईटीडीएमएस कहा जाता है, जोकि इलेक्ट्रॉनिक थेसिस शोध प्रबंध मेटाडेटा मानक है
------
Source:  n ɛ k s ɪ z ʋ ɪ d̪ d̪ ə s ʈ ɛ p s  f ə ɪ m p l ə m ɛ n ʈ eː ʃ ə n  s oː  ɪ s ʈ

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(



--- Inference After Epoch 2.0 ---
Source: d̪ ɪ s  ʋ iː k  j ʉ ʋ ɪ l l ɜː n ə b aw ʈ ʈ ɛ c ɲ ɪ k ə l ɪ n f ɹ ə s ʈ ɹ ə k tʃ ə  a n ɖ ɑː c ɪ ʈ ɛ k tʃ ə ɒ ʋ ɖ ɪ dʒ ɪ ʈ ə l l aj b ɾ i ɪ n ɖ iː ʈ eː l  
Predicted: इस सप्ताह, आप डिजिटल लाइब्रेरी के तकनीकी ढांचे और आर्किटेक्चर के बारे में विस्तार से जानेंगे
Reference: इस हफ्ते, आप डिजिटल लाइब्रेरी की टेक्निकल इंफ्रास्ट्रक्चर और आर्किटेक्चर के बारे में विस्तार से जानेंगे
------
Source: d̪ ə spn h a z ɒː l s oː ɖ ɪ ʋ ɛ l ə p ʈ ə ɲ ʉː m ɛ ʈ ə ɖ eː ʈ ə s ʈ a n ɖ ɪ ɖ  k ɒː l ɖ spn d̪ a ʈ ɪ z  ɛ ʎ ɪ k ʈ ɹ a ɲ ɪ k  spn  ɖ ɪ s ə ʈ eː ʃ ə n m ɛ ʈ ə ɖ eː ʈ ə  s ʈ a n ɖ ɪ ɖ  
Predicted: एलएमएस ने एक नया मेटाडाटा मानक को भी विकसित किया है, जिसे इलेक्ट्रॉनिक फॉर्मेट संचार मेटाडाटा मानक कहा जाता है
Reference: एनडीएलटीडी ने एक नया मेटाडेटा मानक भी विकसित किया है जिसे ईटीडीएमएस कहा जाता है, जोकि इलेक्ट्रॉनिक थेसिस शोध प्रबंध मेटाडेटा मानक है
------
Source:  n ɛ k s ɪ z ʋ ɪ d̪ d̪ ə s ʈ ɛ p s  f ə ɪ m p l ə m ɛ n ʈ eː ʃ ə n  s oː  ɪ s ʈ a b ʎ ɪ ʃ ɪ ŋ  d̪ ə sp

TrainOutput(global_step=20810, training_loss=0.2857225331934755, metrics={'train_runtime': 14739.6179, 'train_samples_per_second': 5.647, 'train_steps_per_second': 1.412, 'total_flos': 2.254627632316416e+16, 'train_loss': 0.2857225331934755, 'epoch': 10.0})